In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
test= [
'1163751742',
'1381373672',
'2136511328',
'3694931569',
'7463417111',
'1319128137',
'1359912421',
'3125421639',
'1293138521',
'2311944581',
]

# A* Algorithm
- brought to you by https://www.redblobgames.com/pathfinding/a-star/introduction.html

In [3]:
import heapq

class PriorityQueue:
    def __init__(self):
        self.elements: list[tuple[float, tuple]] = []
    
    def empty(self) -> bool:
        return not self.elements
    
    def put(self, item: tuple, priority: float):
        heapq.heappush(self.elements, (priority, item))
    
    def get(self) -> tuple:
        return heapq.heappop(self.elements)[1]

In [4]:
class RiskMap:
    def __init__(self, input_grid):
        self._cost_grid =  self._read_grid(input_grid)
        self._end_corner = (len(self._cost_grid)-1,len(self._cost_grid[0])-1)
        self._travel_cost =None
        self._path =None
    
    def _manhatten_dist(self, cur_pt, end=None):
        if end is None:
            end = self._end_corner
        
        return abs(cur_pt[0] - end[0])+abs(cur_pt[1]-end[1])
        
    def _read_grid(self, input_grid):
        cost_grid=[]
        for line in input_grid:
            cost_grid.append([int(i) for i in line.strip()])
        return cost_grid
    
    def _get_neighbors(self, cur_pt):
        add_amounts = [(1,0) , (-1,0), (0,1), (0,-1)]
        (x_lim, y_lim) = self._end_corner
        neighbors = [ tuple([cur_pt[i]+a[i] for i in range(2)]) for a in add_amounts 
                                                        if  ((cur_pt[0]+a[0] >=0) 
                                                            &(cur_pt[1]+a[1] >=0) 
                                                            &(cur_pt[0]+a[0] <=x_lim) 
                                                            &(cur_pt[1]+a[1] <=y_lim) 
                                                           ) ]
                    
        return neighbors
    
    def _get_cost(self,pt):
        return  self._cost_grid[pt[0]][pt[1]] 
    
    def __str__(self):
        """
        I like visuals so this plots out the path :) 
        """
        if self._path is not None:
            full_grid = [ ['   ' for _i in range(self._end_corner[1]+1)] for _j in range(self._end_corner[0]+1)]

            pt = self._end_corner
            full_grid[pt[0]][pt[1]] =  f'{self._travel_cost[pt]:3d}'
            
            while pt != (0,0) :
                pt = self._path[pt]
                full_grid[pt[0]][pt[1]] =  f'{self._travel_cost[pt]:3d}'
                
            out = ['|'.join(row) for row in full_grid]
            return '\n'.join(out)
        
    def generate_path(self, start=(0,0), end=None):
        if end is None:
            end = self._end_corner
        explore = PriorityQueue()
        explore.put(start,0)
        came_from = {start : None}
        cost_so_far = {start : 0}
        
        while not explore.empty():
            cur_pt = explore.get()
            if cur_pt == end:
                break
            for neighbor in self._get_neighbors(cur_pt):
                new_cost = cost_so_far[cur_pt] + self._get_cost(neighbor)
                if neighbor not in cost_so_far.keys() or new_cost < cost_so_far[neighbor]:
                    cost_so_far[neighbor] = new_cost
                    priority = new_cost + self._manhatten_dist(neighbor)
                    explore.put(neighbor, priority)
                    came_from[neighbor] = cur_pt
                    
                    
        self._path = came_from
        self._travel_cost = cost_so_far
        return cost_so_far[end]
                
                
   

In [5]:
class RiskMap5(RiskMap):
    def __init__(self, input_grid):
        RiskMap.__init__(self, input_grid)
        self._end_corner = (len(self._cost_grid)*5-1,len(self._cost_grid[0])*5-1)
        
    def _get_cost(self,pt):
        limits = [x+1 for x in self._end_corner]
        limits = [ x//5 for x in limits ]
    
        fold_copy = [ pt[i]//limits[i] for i in range(2) ]
        
        orig_point = [int(pt[i] - fold_copy[i]*limits[i]) for i in range(2)]
        
        original_cost = self._cost_grid[orig_point[0]][orig_point[1]] 
        # we're only 1-9 digits, so its mod 9 but shifted we need 9 to be 9 and 10 -> 1 
        # so subtract 1 to put on 0-8 scale, add what's needed, take modulo, and add 1 to go back to 1-9 scale
        new_cost = ((original_cost-1) + sum(fold_copy))%9 +1
    
        return  new_cost
        
        

In [6]:
tmap =  RiskMap(test)
t5map =  RiskMap5(test)

In [7]:
tmap.generate_path()
t5map.generate_path()

40

315

In [8]:
int('011111100101',2)

2021

In [9]:
print(tmap)


  0|   |   |   |   |   |   |   |   |   
  1|   |   |   |   |   |   |   |   |   
  3|  4|  7| 13| 18| 19| 20|   |   |   
   |   |   |   |   |   | 21| 26|   |   
   |   |   |   |   |   |   | 27| 28|   
   |   |   |   |   |   |   |   | 31|   
   |   |   |   |   |   |   |   | 33|   
   |   |   |   |   |   |   |   | 36|   
   |   |   |   |   |   |   |   | 38| 39
   |   |   |   |   |   |   |   |   | 40


In [10]:
with open('tmp_test15.txt','w') as f:
    print(t5map, file=f)

In [11]:
prd_data =[]
with open('data/input_15.txt', 'r') as f:
    prd_data = [line.strip() for line in f.readlines()]

In [12]:
prd_map = RiskMap(prd_data)
prd_map_5 = RiskMap5(prd_data)

In [13]:
%%time
prd_map.generate_path()

CPU times: user 87.5 ms, sys: 1.96 ms, total: 89.4 ms
Wall time: 88.9 ms


537

In [14]:
%%time
prd_map_5.generate_path()

CPU times: user 2.41 s, sys: 13.6 ms, total: 2.43 s
Wall time: 2.43 s


2881

In [15]:
with open('tmp.txt','w') as f:
    print(prd_map, file=f)